# Getting Started with Trading Model Pipeline

This notebook will walk you through using the trading model pipeline step by step.

## Setup

Make sure you've installed all requirements:
```bash
pip install -r requirements.txt
```

In [ ]:
# Import modules
import sys
sys.path.append('..')  # Add parent directory to path

from data.collector import DataCollector
from strategies.moving_average import MovingAverageCrossover
from strategies.mean_reversion import MeanReversionZScore
from strategies.momentum import MomentumStrategy
from backtesting.engine import Backtester
import config

print("✓ Modules imported successfully")

## Step 1: Collect Data

In [ ]:
# Initialize data collector
collector = DataCollector(use_cache=True)

# Download data
tickers = ['SPY', 'QQQ', 'GLD']
prices = collector.fetch_data(tickers, '2020-01-01', '2024-12-31')

# Clean and calculate returns
prices = collector.clean_data()
returns = collector.calculate_returns()

# Show summary
collector.get_summary_stats()

## Step 2: Visualize Data

In [ ]:
import matplotlib.pyplot as plt

# Plot normalized prices
normalized = (prices / prices.iloc[0]) * 100

plt.figure(figsize=(12, 6))
normalized.plot(linewidth=2)
plt.title('Normalized Price Performance (Base = 100)', fontsize=14, fontweight='bold')
plt.ylabel('Normalized Price')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Step 3: Create and Test a Strategy

In [ ]:
# Create Moving Average strategy
strategy = MovingAverageCrossover(fast=20, slow=50)

# Generate signals
signals = strategy.generate_signals(prices['SPY'])

# Print strategy info
strategy.print_info()

print("\nLast 10 signals:")
print(signals.tail(10))

## Step 4: Run Backtest

In [ ]:
# Initialize backtester
backtester = Backtester(initial_capital=100000, transaction_cost=0.001)

# Run backtest
results = backtester.run(
    strategy=strategy,
    price_data=prices['SPY'],
    returns_data=returns['SPY']
)

## Step 5: Visualize Results

In [ ]:
# Plot backtest results
backtester.plot_results()

## Step 6: Compare Multiple Strategies

In [ ]:
# Create multiple strategies
strategies = [
    MovingAverageCrossover(fast=20, slow=50),
    MeanReversionZScore(window=20, threshold=2.0),
    MomentumStrategy(lookback=20)
]

# Backtest each
all_results = []

for strat in strategies:
    bt = Backtester(initial_capital=100000, transaction_cost=0.001)
    result = bt.run(strat, prices['SPY'], returns['SPY'])
    all_results.append(result)
    print("\n" + "-"*70 + "\n")

In [ ]:
# Compare strategies
import pandas as pd

comparison = pd.DataFrame({
    'Strategy': [r['strategy_name'] for r in all_results],
    'Total Return': [f"{r['metrics']['Total Return']:.2%}" for r in all_results],
    'CAGR': [f"{r['metrics']['CAGR']:.2%}" for r in all_results],
    'Sharpe': [f"{r['metrics']['Sharpe Ratio']:.3f}" for r in all_results],
    'Max DD': [f"{r['metrics']['Max Drawdown']:.2%}" for r in all_results],
    'Win Rate': [f"{r['metrics']['Win Rate']:.2%}" for r in all_results]
})

print("\nSTRATEGY COMPARISON")
print("=" * 100)
print(comparison.to_string(index=False))
print("=" * 100)

In [ ]:
# Plot cumulative returns comparison
plt.figure(figsize=(14, 7))

for result in all_results:
    plt.plot(result['cumulative_returns'], 
            label=result['strategy_name'], linewidth=2)

plt.title('Strategy Comparison: Cumulative Returns', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Next Steps

1. **Experiment with parameters**: Change MA periods, z-score thresholds, etc.
2. **Try different assets**: Test on different tickers
3. **Add new strategies**: Create your own in the `strategies/` folder
4. **Optimize**: Use grid search to find best parameters
5. **Go live**: Connect to Alpaca API for paper trading

---

**Happy trading!** 🚀